In [ ]:
import os

from course_recommender.extraction_schema import StudentPreferences
from course_recommender import extraction

EXAMPLE_INPUT_ENGINEERING = "I am in my 7th semester studying Mechanical Engineering at RWTH Aachen University. I have taken elective courses in Thermodynamics, Robotics, and Renewable Energy Systems. Robotics was the most fascinating course for me, and I plan to focus on this area for my master's degree. Additionally, I have a keen interest in Automation and Control Systems, especially in the context of Smart Manufacturing and IoT. However, I am not very enthusiastic about subjects that delve deeply into theoretical physics or advanced calculus. I prefer courses that involve hands-on projects and the application of engineering principles to real-world problems."

EXAMPLE_INPUT_MED = "I am currently in my 5th semester studying Medicine at the University of Munich. So far, I have completed elective courses in Human Anatomy, Pharmacology, and Medical Ethics. My favorite course was Pharmacology, and I am eager to pursue a specialization in this area for my postgraduate studies. I am also very interested in Neuroscience, particularly in Neurodegenerative Diseases and their treatments. On the other hand, I am not very fond of courses that focus heavily on molecular biology or genetics. I enjoy practical courses that involve patient interaction and clinical practices."

EXAMPLE_INPUT_CS = "Im a student in my 6th semester currently studying Computer Science at TUM. I already did the following electory courses: ERDB, IT Securiy and Business Analytics and Machine Learning. I really liked Machine Learning and I want to specialize in this subject for my masters. Im also interested in System Design especially in Microservice and Cloud Architecture. I don't like low level programming such as C. I like high level languages such as Java and Python"

student_preferences: StudentPreferences = extraction.extract_student_preferences(EXAMPLE_INPUT_MED)

print(student_preferences)

In [ ]:
import sqlite3

# Extract school
module_con = sqlite3.connect(os.getenv("DB_PATH"))
if student_preferences.school:
    school_id = module_con.execute("SELECT org_id FROM organisations WHERE name = ? and org_type = 'school'",
                                   (student_preferences.school,)).fetchone()[0]
else:
    school_id = None
    
print(f"{school_id}: {student_preferences.school}")

In [ ]:
from course_recommender.topic_mapper import vector_similarity_search, build_index

index_with_topics = build_index(school_id=school_id)
topic_mappings = []
for topic in student_preferences.topics_of_interest:
    topics = vector_similarity_search(index_with_topics=index_with_topics, topic_str=topic, k=5)
    topic_mappings += topics

print(topic_mappings)

In [ ]:
topic_ids = [topic_id for topic_id, _ in topic_mappings]
cursor = module_con.cursor()

# Generate a placeholder string for the IN clause
placeholders = ', '.join(['?'] * len(topic_ids))

# Use the placeholders in the query
query = f"SELECT m.module_id, m.name, count(m.org_id) as c FROM modules m, module_topic_mappings mtm WHERE m.module_id = mtm.module_id and mtm.topic_id IN ({placeholders}) group by m.module_id, m.name order by c desc LIMIT 10"

# Execute the query with the topic_ids as parameters
cursor.execute(query, topic_ids).fetchall()
